# Estimating Distance to Galaxies

In this exercise, you will use the SDSS (Sloan Digital Sky Survey) database to retrieve images of selected galaxies and measure their angular sizes. We will again utilize the **Astroquery** Python package for retrieving data from the SDSS. Once the data is plotted, you will interactively measure the angular size and convert that to a distance given our assumptions.

## Retrieving SDSS Images with Astroquery

**Astroquery** is a Python library designed to simplify the process of querying astronomical databases. It allows you to retrieve data from various astronomical surveys, including SDSS.

First, we'll ensure we have the needed libraries installed:

In [ ]:
%%capture
!pip install astroquery bokeh ipywidgets

### Step 1. Import Libraries

We'll import the necessary libraries below:

In [ ]:
from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord
from astropy.visualization import make_lupton_rgb
from astropy.wcs import WCS
from astropy.nddata import Cutout2D
from bokeh.plotting import figure, show, output_notebook
import astropy.units as u
from bokeh.models import ColumnDataSource, PolyDrawTool, CustomJS
import numpy as np
import ipywidgets as w
from IPython.display import display

output_notebook()

### Step 2. Query the SDSS Database

Using Astroquery, you can search for images of the galaxies you are interested in. You will need to know the coordinates of each galaxy (right ascension and declination). After querying the database, you can download the image:

In [ ]:
# Replace these coordinates with the ones for your galaxies
position = SkyCoord(2.02345833, 14.83980556, unit="deg")

# Query SDSS for image data in g, r, i bands
bands = ['g', 'r', 'i']
images = {band: SDSS.get_images(coordinates=position, band=band)[0][0] for band in bands}

# Create cutouts around the galaxy's position for each band
cutouts = {band: Cutout2D(images[band].data, position=position, size=(200, 200), wcs=WCS(images[band].header)) for band in bands}

# Initial RGB image
rgb_image = make_lupton_rgb(cutouts['i'].data, cutouts['r'].data, cutouts['g'].data, stretch=0.5, Q=10)

In [ ]:
# Convert the RGB image to a 2D array of RGBA values
def rgb_to_rgba(image):
    # Create an empty array for RGBA values
    rgba_image = np.empty((image.shape[0], image.shape[1]), dtype=np.uint32)
    rgba_view = rgba_image.view(dtype=np.uint8).reshape((image.shape[0], image.shape[1], 4))
    
    # Assign RGB values
    rgba_view[:, :, :3] = image
    
    # Set alpha channel to fully opaque
    rgba_view[:, :, 3] = 255
    
    # Convert to a format Bokeh can handle (1D array of uint32)
    # rgba_image = (rgba_image[:, :, 0] << 24) | (rgba_image[:, :, 1] << 16) | (rgba_image[:, :, 2] << 8) | rgba_image[:, :, 3]
    # rgba_image = rgba_image.view(np.uint32).reshape((image.shape[0], image.shape[1], 4))
    
    return rgba_image

rgba_image = rgb_to_rgba(rgb_image)

This code will retrieve data related to the galaxy's position. Adjust the parameters according to the specific galaxies you are studying and provide you with the image data that you can then use for further analysis.

### Step 3. Measure Angular Size

Measure the angular size of the detected galaxy. Utilize the "Polygon Draw Tool" from Bokeh to draw a line across the galaxy and calculate the angular size based on the drawn line. The angular size is calculated based on the pixel scale of the image and the length of the line drawn, and will be displayed in the widget below the image in units of `degrees`.

In [ ]:
# Create a Bokeh figure to display the galaxy image
p = figure(title="Galaxy Image")
p.image_rgba(image=[rgba_image], x=0, y=0, dw=rgb_image.shape[0], dh=rgb_image.shape[1])

# Create a ColumnDataSource for the line
line_source = ColumnDataSource(data=dict(xs=[[]], ys=[[]]))

# Add a line glyph
line_renderer = p.multi_line(xs='xs', ys='ys', source=line_source, line_color='red', line_width=2)

# Add the PolyDrawTool and PolyEditTool to the plot
poly_draw_tool = PolyDrawTool(renderers=[line_renderer], num_objects=1)
p.add_tools(poly_draw_tool)
p.toolbar.active_drag = poly_draw_tool

# Convert WCS pixel scale matrix to degrees
wcs = WCS(images['r'].header)
pixel_scale_matrix_x, pixel_scale_matrix_y = wcs.proj_plane_pixel_scales()[0].to(u.deg).value, wcs.proj_plane_pixel_scales()[1].to(u.deg).value

# JavaScript code to update the angular size
callback_code = """
const data = source.data;
const xs = data['xs'][0];
const ys = data['ys'][0];

if (xs.length == 2 && ys.length == 2) {
    const dx = xs[1] - xs[0];
    const dy = ys[1] - ys[0];
    const length_pixels = Math.sqrt(dx * dx + dy * dy);
    const angular_size = Math.sqrt((dx * scale_x) ** 2 + (dy * scale_y) ** 2);

    // Update the ipywidgets FloatText widget
    const widget = document.querySelector("input[type='number']");
    if (widget) {{
        widget.value = angular_size;
    }}
}
"""

# Add a CustomJS callback to update angular size
callback = CustomJS(args=dict(source=line_source, 
                              scale_x=pixel_scale_matrix_x, 
                              scale_y=pixel_scale_matrix_y), code=callback_code)

line_source.js_on_change('data', callback)

In [ ]:
# Display the plot
show(p)

# Angular size widget
angular_size_widget = w.FloatText(disabled=True)
angular_size_label = w.Label(value="Angular Size (degrees):")
display(w.HBox([angular_size_label, angular_size_widget]))

### Step 4. Calculate the Distance

Using the equation learned previously, calculate the distance to the galaxy:

$$ d = \frac{D}{\theta} $$

In [ ]:
# Enter your angular size (in degrees) from above
angular_size = 0.007879115340206641
angular_size_in_radians = np.radians(angular_size)

diameter_of_milky_way = 0.0307  # in Mpc

distance = diameter_of_milky_way / angular_size_in_radians

print(f"The estimated distance to the galaxy is approximately {distance:.2f} Mpc.")

## Using Your Own Galaxies

As before, return to the top of this notebook and input the coordinates of each of your galaxies. RE-run the cells and calculate the distances. Be sure to record your results in the [Google Sheet](https://docs.google.com/spreadsheets/d/1mVRovTF8C1UJkQakQG_E9emnpJIRb1rdNx6UPNcm3rs/edit?usp=sharing).